## 2.7 Machine Learning for Ranking with Spark MLLIB

First we need to load our training data into pandas dataframe.  The data is in tab separated formatand the easiest way to load this is using the ```pandas``` ```read_table()``` method. 

In [33]:
import csv
import pandas as pd

df = pd.read_table("data/fullDataset.tsv",header=0)

Let's check out the shape and column names:

In [34]:
print(df.shape)
print(df.columns)

(78500, 19)
Index([u'key', u'query', u'Title', u'LeafCats', u'ItemID', u'X_unit_id',
       u'SCORE', u'label_relevanceGrade', u'label_relevanceBinary',
       u'feature_1', u'feature_2', u'feature_3', u'feature_4', u'feature_5',
       u'feature_6', u'feature_7', u'feature_8', u'feature_9', u'feature_10'],
      dtype='object')


Here is a sample of the data:

In [35]:
df.sample(5)

,key,query,Title,LeafCats,ItemID,X_unit_id,SCORE,label_relevanceGrade,label_relevanceBinary,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
4259,6619,flieger type b,Montre pilote FLIEGER vintage style Seagull ty...,31387,3255,724907848,3:3:,6,1,2676.618000,0,-7.509815,-4.035674,37.775288,0.442953,0,0,173,-100.000000
76131,151375,supreme international,SUPREME 2015 F/W CDG BOX LOGO WORLDWIDE INTERN...,15687,54978,780087414,3:3:3,6,1,3144.944336,1,-8.793618,-4.031732,24.298122,0.587773,347,-1000000,138,-100.000000
47480,154492,volkswagen,Volkswagen : Beetle-New 2.5 Convertible 2-Door,31876,56446,780086605,3:3:3,6,1,2732.750000,1,-8.174521,-3.379251,15.431207,0.043624,487446,50295,143,-100.000000
15594,15128,bruce sutter autograph,BRUCE SUTTER 2004 Topps Chrome Certified Aut...,213,7544,724911220,3:3:3,6,1,3004.083252,1,-8.006219,-4.861247,72.422882,0.463425,87,-1000000,184,2.138143
38034,164943,adidas size 9,Adidas High Top Men's Size 9 Basketball Shoes ...,15709,61587,809745982,3:3:3,6,1,3473.000000,1,-7.390667,-4.736623,33.012997,0.473685,8560,-1000000,141,-100.000000


The columns are:

Column name             | Description
------------------------|-----------------------------------------------------------------------
key                     |  Used to join back to the original dataset and add any additional fields as needed
query                   |  Un-normalized query keywords (without user constraints)
Title                   |  Un-normalized title
LeafCats                |  Item leaf category IDs
ItemID                  |  Anonymized itemID. This is not the actual item ID
X_unit_id               |  The query ID used for grouping query-item pairs by their search, primarily for per-query metrics. Essentially it's a "search ID". We can also group by query or normalized query
SCORE                   |  The scores given by up to three judges characterizing the relevance/relevance problem of the query-item pair
label_relevanceGrade    |  The SCORE averaged and rounded and converted to a relevance graded judgment 0-6, 6 being the best. Note this is very approximate
label_relevanceBinary   |  The SCORE converted to a binary relevant(==1) or not relevant (==0) judgment.  This is a more accurate label than the Grade, I recommend it as a training target

Features (In brief): 

* query features: feature_2, feature_7, feature_8
* item features: feature_3, feature_4, feature_9
* query-item features: feature_1, feature_5, feature_6, feature_10

## Getting ready for Machine Learning

We start by simply exploring how we might classify queries as relevent / not relevant.  We will explore a series of different models to do this. The first is logistic regression, we will also use SVM and finally classification tree's.  Along the way we will look at over fitting / generalization and how to evaluate models.  

We can do all this using Spark MLLIB - first we have to findspark and get our spark context:


In [36]:
import findspark
import os
findspark.init(os.getenv('HOME') + '/spark-1.6.0-bin-hadoop2.6')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

In [37]:
import pyspark
try: 
    print(sc)
except NameError:
    sc = pyspark.SparkContext()
    print(sc)

It is easy to load the tsv data into a Spark DataFrame:

In [38]:
from pyspark.sql import SQLContext
import os

sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options() \
        .options(header='true', inferSchema='true', delimiter='\t') \
        .load(os.getcwd() + '/data/fullDataset.tsv') 
        
df.schema


StructType(List(StructField(key,IntegerType,true),StructField(query,StringType,true),StructField(Title,StringType,true),StructField(LeafCats,StringType,true),StructField(ItemID,IntegerType,true),StructField(X_unit_id,IntegerType,true),StructField(SCORE,StringType,true),StructField(label_relevanceGrade,IntegerType,true),StructField(label_relevanceBinary,IntegerType,true),StructField(feature_1,DoubleType,true),StructField(feature_2,IntegerType,true),StructField(feature_3,DoubleType,true),StructField(feature_4,DoubleType,true),StructField(feature_5,DoubleType,true),StructField(feature_6,DoubleType,true),StructField(feature_7,IntegerType,true),StructField(feature_8,DoubleType,true),StructField(feature_9,IntegerType,true),StructField(feature_10,DoubleType,true)))

Now we can extract the features and the target for the machine learning algorithms:

In [39]:
sqlContext.registerDataFrameAsTable(df,'dataset')
sqlContext.tableNames()

data_full = sqlContext.sql("select label_relevanceBinary, feature_1, feature_2, feature_3, feature_4 \
                       feature_5, feature_6, feature_7, feature_8, feature_9, feature_10 \
               from dataset").rdd


We also split the data into test and validation data sets - splitting 75%:25% between the training and test sets:

In [40]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parseRecord(line):
    return LabeledPoint(line[0], line[1:])

data = data_full.map(parseRecord)

#features = data_full.map(lambda x: x.feature_1)
from pyspark.mllib.feature import StandardScaler
scaler1 = StandardScaler().fit(data)


data_train, data_test = data.randomSplit([0.75,0.25])

Py4JJavaError: An error occurred while calling o416.fitStandardScaler.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 158.0 failed 1 times, most recent failure: Lost task 0.0 in stage 158.0 (TID 158, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/csumb/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/csumb/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/csumb/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/csumb/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 77, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))
Typ

In [31]:
print('Training data records = ' + str(data_train.count()))
print('Training data records = ' + str(data_test.count()))

data_train.take(5)

Training data records = 58615
Training data records = 19795


[LabeledPoint(0.0, [2676.618,0.0,-9.020574,-5.221152,0.0,241.0,-1000000.0,139.0,-100.0]),
 LabeledPoint(1.0, [2676.618,0.0,-8.599997,-5.31775,0.805351,0.0,0.0,150.0,2.53809117973]),
 LabeledPoint(0.0, [3428.06665,1.0,-8.23653,-3.406829,0.0561681033807,742.0,-1000000.0,139.0,-100.0]),
 LabeledPoint(1.0, [2676.618,0.0,-7.454222,-4.874171,0.051492,338.0,-1000000.0,213.0,0.441697325035]),
 LabeledPoint(1.0, [2676.618,0.0,-7.52216,-4.38311,0.15502195947,0.0,0.0,141.0,-100.0])]

## Fitting an SVM - a simple classifier

In [15]:
model = SVMWithSGD.train(data_train.map(parseRecord), iterations=100)

In [16]:
model

(weights=[1935.22521984,2.35665977665,0.779288560111,2.09313254781,0.850439376318,3670.3593062,-2662.2170945,-14.1656243555,54.2181876912], intercept=0.0)

In [17]:
# Evaluating the model on test data
preds = data_test.map(parseRecord).map(lambda p: (p.label, model.predict(p.features)))
err = preds.filter(lambda (v, p): v != p).count() / float(data_test.count())
print("Training Error = " + str(err))

Training Error = 0.453355824771


## Fitting Logistic Regression

In [18]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel

model = LogisticRegressionWithLBFGS.train(data_train.map(parseRecord))

In [19]:
model

(weights=[0.000438641825962,0.643992236566,0.0160266941194,0.229458388736,1.18529259753,3.67571490154e-07,-7.292236451e-08,-0.000353467229084,0.000143238868138], intercept=0.0)

In [20]:
# Evaluating the model on training data
preds = data_test.map(parseRecord).map(lambda p: (p.label, model.predict(p.features)))
err = preds.filter(lambda (v, p): v != p).count() / float(data_test.count())
print("Training Error = " + str(err))

Training Error = 0.300414046925


## Normalise the features

In the previous sections we used the raw feature scores.  We should normalize the data first.


In [21]:
from pyspark.mllib.feature import StandardScaler
features = data.map(lambda x: x.features)
scaler1 = StandardScaler().fit(features)

NameError: name 'data' is not defined